# Li(110) Step-Edge Adatom Diffusion – Restartable NEB Workflow (LCAO)

This notebook is structured to be **restartable**:

- Intermediate structures (step slab, initial/final, relaxed states, NEB images) are saved to `.traj` files.
- On rerun, the notebook **reads from disk if files already exist**, skipping expensive rebuild/relax steps.
- Only the **surface construction** differs from a flat Li(110): here we build a monoatomic step and place the adatom on upper/lower terraces away from the box edges.


In [ ]:
# Cell 0: Imports & global config

from ase.build import bcc110
from ase import Atoms, Atom
from ase.constraints import FixAtoms
from ase.optimize import BFGS
from ase.mep import NEB
from ase.io import read, write

import numpy as np
import os

import nglview as nv

from gpaw import GPAW, LCAO

# Global settings
a0 = 3.49  # Li bcc lattice constant (adjust if needed)
output_dir = "Li110_step_edge_neb_restartable"
os.makedirs(output_dir, exist_ok=True)

# Paths for restartable artifacts
slab_path            = os.path.join(output_dir, "step_slab.traj")
init_unrelaxed_path  = os.path.join(output_dir, "initial_unrelaxed.traj")
final_unrelaxed_path = os.path.join(output_dir, "final_unrelaxed.traj")
init_relaxed_path    = os.path.join(output_dir, "initial_relaxed.traj")
final_relaxed_path   = os.path.join(output_dir, "final_relaxed.traj")
neb_images_path      = os.path.join(output_dir, "neb_images.traj")


In [13]:
# Cell 1: Build or load flat Li(110) slab (orthogonal cell, practical size)

if os.path.exists(slab_path):
    print("Loading existing step slab from", slab_path)
    step_slab = read(slab_path)
else:
    print("Creating new flat Li(110) slab and carving step...")
    # size = (nx, ny, nz); ny must be even for orthogonal=True
    slab_flat = bcc110('Li', a=a0, size=(4, 4, 6), vacuum=8.0, orthogonal=True)
    slab_flat.center(axis=2)  # center along z

    pos = slab_flat.get_positions()
    x = pos[:, 0]
    y = pos[:, 1]
    z = pos[:, 2]

    # Identify distinct z-layers
    z_unique = np.unique(np.round(z, 5))
    z_unique.sort()
    z_top = z_unique[-1]        # topmost layer
    z_second = z_unique[-2]     # layer just below (for lower terrace)

    tol = 1e-3
    mask_top = np.abs(z - z_top) < tol

    # Define the step edge by a plane in y (left vs right half of surface)
    y_cut = np.mean(y[mask_top])

    # Remove top-layer atoms on "left" half (y < y_cut)
    keep_mask = ~(mask_top & (y < y_cut))
    step_slab = slab_flat[keep_mask]
    step_slab.center(axis=2)

    # Save metadata needed later
    np.savez(os.path.join(output_dir, "step_meta.npz"),
             z_unique=z_unique, z_top=z_top, z_second=z_second, y_cut=y_cut)

    write(slab_path, step_slab)
    print("Step slab written to", slab_path)

step_slab


Loading existing step slab from Li110_step_edge_neb_restartable/step_slab.traj


Atoms(symbols='Li88', pbc=[True, True, False], cell=[13.96, 9.871210665364204, 28.339013331705253], tags=...)

In [14]:
# Cell 2: Load step metadata (z_top, z_second, y_cut)

import numpy as _np
meta_file = os.path.join(output_dir, "step_meta.npz")
if not os.path.exists(meta_file):
    # Recompute if missing (e.g. slab loaded from elsewhere)
    pos = step_slab.get_positions()
    x = pos[:, 0]
    y = pos[:, 1]
    z = pos[:, 2]
    z_unique = _np.unique(_np.round(z, 5))
    z_unique.sort()
    z_top = z_unique[-1]
    z_second = z_unique[-2]
    tol = 1e-3
    mask_top = _np.abs(z - z_top) < tol
    y_cut = _np.mean(y[mask_top])
    _np.savez(meta_file, z_unique=z_unique, z_top=z_top, z_second=z_second, y_cut=y_cut)
else:
    data = _np.load(meta_file)
    z_unique = data["z_unique"]
    z_top = float(data["z_top"])
    z_second = float(data["z_second"])
    y_cut = float(data["y_cut"])

print("z layers:", z_unique)
print(f"z_top = {z_top:.4f}, z_second = {z_second:.4f}")
print(f"y_cut = {y_cut:.4f}")


z layers: [ 8.      10.4678  12.93561 15.40341 17.87121 20.33901]
z_top = 20.3390, z_second = 17.8712
y_cut = 3.7017


In [15]:
# Cell 3: Visualise step-edge slab

view_step = nv.show_ase(step_slab)
view_step.add_unitcell()
view_step


NGLWidget()

In [16]:
# Cell 4: Helper functions – centered initial and final adatom positions

def make_initial(step_slab, y_cut, z_top, height=2.0, edge_fraction=0.3):
    """Adatom on UPPER terrace, near step edge but shifted toward interior."""
    atoms = step_slab.copy()
    pos = atoms.get_positions()
    x, y, z = pos[:,0], pos[:,1], pos[:,2]

    tol = 1e-3
    # upper terrace = top layer, y > y_cut
    mask_upper = (np.abs(z - z_top) < tol) & (y > y_cut)
    if not np.any(mask_upper):
        raise RuntimeError("No atoms found on upper terrace (check y_cut / z_top).")

    x_u = x[mask_upper]
    y_u = y[mask_upper]

    x_min, x_max = x_u.min(), x_u.max()
    y_min, y_max = y_u.min(), y_u.max()

    # x at centre of terrace
    x_new = 0.5 * (x_min + x_max)

    # y_new between step edge and terrace interior
    y_target = 0.5 * (y_min + y_max)
    y_new = y_cut + edge_fraction * (y_target - y_cut)

    z_new = z_top + height

    atoms.append(Atom('Li', (x_new, y_new, z_new)))
    return atoms


def make_final(step_slab, y_cut, z_second, height=2.0, edge_fraction=0.3):
    """Adatom on LOWER terrace, near step edge but shifted toward interior."""
    atoms = step_slab.copy()
    pos = atoms.get_positions()
    x, y, z = pos[:,0], pos[:,1], pos[:,2]

    tol = 1e-3
    # lower terrace = second layer, y < y_cut
    mask_lower = (np.abs(z - z_second) < tol) & (y < y_cut)
    if not np.any(mask_lower):
        raise RuntimeError("No atoms found on lower terrace (check y_cut / z_second).")

    x_l = x[mask_lower]
    y_l = y[mask_lower]

    x_min, x_max = x_l.min(), x_l.max()
    y_min, y_max = y_l.min(), y_l.max()

    # x at centre
    x_new = 0.5 * (x_min + x_max)

    # y_new between step edge and terrace interior (y_target < y_cut)
    y_target = 0.5 * (y_min + y_max)
    y_new = y_cut + edge_fraction * (y_target - y_cut)

    z_new = z_second + height

    atoms.append(Atom('Li', (x_new, y_new, z_new)))
    return atoms


In [17]:
# Cell 5: Build or load unrelaxed initial and final configurations

if os.path.exists(init_unrelaxed_path) and os.path.exists(final_unrelaxed_path):
    print("Loading unrelaxed initial/final from disk...")
    initial_unrelaxed = read(init_unrelaxed_path)
    final_unrelaxed = read(final_unrelaxed_path)
else:
    print("Creating unrelaxed initial/final...")
    initial_unrelaxed = make_initial(step_slab, y_cut, z_top, height=2.0, edge_fraction=0.3)
    final_unrelaxed   = make_final(step_slab, y_cut, z_second, height=2.0, edge_fraction=0.3)

    write(init_unrelaxed_path, initial_unrelaxed)
    write(final_unrelaxed_path, final_unrelaxed)
    print("Unrelaxed states written to", init_unrelaxed_path, "and", final_unrelaxed_path)

initial_unrelaxed, final_unrelaxed


Loading unrelaxed initial/final from disk...


(Atoms(symbols='Li89', pbc=[True, True, False], cell=[13.96, 9.871210665364204, 28.339013331705253], tags=...),
 Atoms(symbols='Li89', pbc=[True, True, False], cell=[13.96, 9.871210665364204, 28.339013331705253], tags=...))

In [18]:
# Cell 6: Visualise unrelaxed INITIAL configuration

view_initial_unrelaxed = nv.show_ase(initial_unrelaxed)
view_initial_unrelaxed.add_unitcell()
view_initial_unrelaxed


NGLWidget()

In [19]:
# Cell 7: Visualise unrelaxed FINAL configuration

view_final_unrelaxed = nv.show_ase(final_unrelaxed)
view_final_unrelaxed.add_unitcell()
view_final_unrelaxed


NGLWidget()

In [20]:
# Cell 8: GPAW helper functions and relaxation routine

def apply_bottom_fix(slab, fraction=0.5):
    """Fix atoms in the bottom 'fraction' of the slab (by z)."""
    slab = slab.copy()
    z = slab.positions[:, 2]
    z_min, z_max = z.min(), z.max()
    z_cut = z_min + fraction * (z_max - z_min)
    mask = z < z_cut
    slab.set_constraint(FixAtoms(mask=mask))
    return slab


def make_gpaw(label, txt_dir=None, mode="LCAO"):
    """Construct a GPAW calculator (LCAO by default)."""
    if txt_dir is None:
        txt_dir = output_dir
    os.makedirs(txt_dir, exist_ok=True)
    if mode.lower() == "lcao":
        calc = GPAW(
            mode=LCAO(),
            xc="PBE",
            kpts=(2, 2, 1),  # relax to (2,1,1) or (1,1,1) for quick tests if needed
            occupations={'name': 'fermi-dirac', 'width': 0.1},
            txt=os.path.join(txt_dir, f"{label}.txt"),
        )
    else:
        from gpaw import PW
        calc = GPAW(
            mode=PW(200),
            xc="PBE",
            kpts=(2, 2, 1),
            occupations={'name': 'fermi-dirac', 'width': 0.1},
            txt=os.path.join(txt_dir, f"{label}.txt"),
        )
    return calc


def relax_state(atoms, label, traj_path, fmax=0.05, maxsteps=200):
    """Relax a configuration with GPAW + BFGS, restartable via traj file."""
    # If a relaxed traj already exists, just read last image
    if os.path.exists(traj_path):
        from ase.io import Trajectory
        print(f"Reading existing relaxed state from {traj_path}")
        with Trajectory(traj_path) as tr:
            relaxed = tr[-1]
        energy = relaxed.get_potential_energy()
        print(f"{label} (from disk): E = {energy:.6f} eV")
        return relaxed, energy

    atoms = apply_bottom_fix(atoms, fraction=0.5)
    atoms.calc = make_gpaw(label)
    dyn = BFGS(atoms, trajectory=traj_path, logfile=os.path.join(output_dir, f"{label}_opt.log"))
    dyn.run(fmax=fmax, steps=maxsteps)
    energy = atoms.get_potential_energy()
    print(f"{label}: E = {energy:.6f} eV")
    return atoms, energy


In [21]:
# Cell 9: Relax initial and final states (restartable)

initial_relaxed, E_i = relax_state(initial_unrelaxed,
                                   label="Li110_step_initial",
                                   traj_path=init_relaxed_path)

final_relaxed,   E_f = relax_state(final_unrelaxed,
                                   label="Li110_step_final",
                                   traj_path=final_relaxed_path)

E_i, E_f


Reading existing relaxed state from Li110_step_edge_neb_restartable/initial_relaxed.traj
Li110_step_initial (from disk): E = -141.373692 eV
Li110_step_final: E = -143.861836 eV


(-141.37369156849587, np.float64(-143.86183606194155))

In [22]:
# Cell 10: Visualise relaxed INITIAL configuration

view_initial_relaxed = nv.show_ase(initial_relaxed)
view_initial_relaxed.add_unitcell()
view_initial_relaxed


NGLWidget()

In [23]:
# Cell 11: Visualise relaxed FINAL configuration

view_final_relaxed = nv.show_ase(final_relaxed)
view_final_relaxed.add_unitcell()
view_final_relaxed


NGLWidget()

In [24]:
# Cell 12: Build or load NEB images (unrelaxed path between relaxed endpoints)

from ase.io import Trajectory

if os.path.exists(neb_images_path):
    print("Loading NEB images from", neb_images_path)
    images = list(Trajectory(neb_images_path))
else:
    print("Creating new NEB images...")
    n_images = 5  # total images including endpoints
    images = [initial_relaxed]
    for i in range(n_images - 2):
        images.append(initial_relaxed.copy())
    images.append(final_relaxed)

    neb = NEB(images)
    neb.interpolate()

    # Save all images to a trajectory for restart
    write(neb_images_path, images)
    print("NEB images written to", neb_images_path)

images


Creating new NEB images...
NEB images written to Li110_step_edge_neb_restartable/neb_images.traj


/var/folders/0d/pk580grx2jxfwhj5nvvws7y00000gn/T/ipykernel_24788/125932124.py:16: FutureWarning: Please import NEB from ase.mep, not ase.neb.
  neb = NEB(images)


[Atoms(symbols='Li89', pbc=[True, True, False], cell=[13.96, 9.871210665364204, 28.339013331705253], tags=..., constraint=FixAtoms(indices=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]), calculator=SinglePointCalculator(...)),
 Atoms(symbols='Li89', pbc=[True, True, False], cell=[13.96, 9.871210665364204, 28.339013331705253], tags=..., constraint=FixAtoms(indices=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47])),
 Atoms(symbols='Li89', pbc=[True, True, False], cell=[13.96, 9.871210665364204, 28.339013331705253], tags=..., constraint=FixAtoms(indices=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 4

In [25]:
# Cell 13: Attach calculators to NEB images

for i, img in enumerate(images):
    label = f"Li110_step_neb_img{i}"
    img.calc = make_gpaw(label)


In [26]:
# Cell 14: Run NEB optimisation (can be rerun; trajectory accumulates)

from ase.neb import NEB
from ase.io import Trajectory

neb = NEB(images)
neb_dyn = BFGS(neb,
               trajectory=os.path.join(output_dir, "Li110_step_neb.traj"),
               logfile=os.path.join(output_dir, "Li110_step_neb.log"))
neb_dyn.run(fmax=0.10)


/var/folders/0d/pk580grx2jxfwhj5nvvws7y00000gn/T/ipykernel_24788/4283266766.py:6: FutureWarning: Please import NEB from ase.mep, not ase.neb.
  neb = NEB(images)


np.True_

In [27]:
# Cell 15: Analyse NEB barrier and visualise NEB path

from ase.neb import NEBTools

nebtools = NEBTools(images)
E_max, dE = nebtools.get_barrier()
print(f"Barrier height (relative to minimum): {E_max:.4f} eV")
print(f"Barrier from initial state: {dE:.4f} eV")

view_neb = nv.show_ase(images)
view_neb.add_unitcell()
view_neb


/var/folders/0d/pk580grx2jxfwhj5nvvws7y00000gn/T/ipykernel_24788/684234765.py:5: FutureWarning: Please import NEBTools from ase.mep, not ase.neb.
  nebtools = NEBTools(images)


Barrier height (relative to minimum): 0.0000 eV
Barrier from initial state: -2.4881 eV


AttributeError: 'list' object has no attribute 'write'